<a href="https://colab.research.google.com/github/torkralle/colab-overtone/blob/master/vocoder_time_invariant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import sys
sys.path.append('/content/drive/MyDrive/Laboratory/Colab/overtone/modules')
import wave_file as wav
import window_function as wf
import average_function as af
import furie_function as ff
import filter

In [2]:
plain_sound_path = '/content/drive/MyDrive/Laboratory/Colab/overtone/sounds/plain.wav'
fs, s0 = wav.wave_read_16bit_mono(plain_sound_path)

overtone_path = '/content/drive/MyDrive/Laboratory/Colab/overtone/sounds/overtone.wav'
fs, s1 = wav.wave_read_16bit_mono(overtone_path)

定数

In [3]:
length_of_s0 = len(s0)
length_of_s1 = len(s1)

N = 4096

band_width = 8
number_of_band = int(N / 2 / band_width)
number_of_frame = int((length_of_s0 - N / 2) / (N / 2))

In [4]:
s2 = np.zeros(length_of_s0)
s1 = filter.highpass_filter(s1)

x = np.zeros(N)
b = np.zeros(N)
w = wf.hanning_window(N)
Y = np.zeros(N, dtype = np.complex)


offset = 48000
for n in range(N):
    b[n] = s1[offset + n] * w[n]

B = np.abs(np.fft.fft(b, N))

for band in range(number_of_band):
    offset = band_width * band
    for k in range(band_width):
        B[offset + k] = af.get_average(band_width, offset, B)

B = ff.symmetric_copy(B)

for frame in range(number_of_frame):
    offset = int(N / 2) * frame
    for n in range(N):
        x[n] = s0[offset + n] * w[n]

    X = np.fft.fft(x, N)
    Y = ff.convolution(X, B)
    y = np.real(np.fft.ifft(Y, N))
    
    offset = int(N / 2) * frame
    for n in range(N):
        s2[offset + n] += y[n]

master_volume = 0.5
s2 /= np.max(np.abs(s2))
s2 *= master_volume

wav.wave_write_16bit_mono(fs, s2, 'changed_filter.wav')

/content/drive/MyDrive/Laboratory/Colab/overtone/modules/furie_function.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  Y[k] = V[k] * B[k]
